In [2]:
from abc import ABC

In [27]:
import time

class Platform:
    def __init__(self, algos, tables):
        self.algos = algos
        self.tables = tables
        self.results = []

    def compress_table(self):
        for algo in self.algos:
            for table in self.tables:
                self.run(algo, table)

    def run(self, algo, table):

        compressed_data = algo.compression(table)
        #start_time, compressed_data = algo.compression(table) # start_time in .compression()? why?

        compression_rate = len(compressed_data) / len(table) # TODO: replace len(table) with a proper binary size

        decompressed_data = algo.decompression(compressed_data)
        loss_rate = (table - decompressed_data) / table # TODO: replace with a separate method
        result = {
            "algorithm": algo.name,
            "table": table,
            "compression_rate": compression_rate,
            "compression_time": 0,#compression_time,
            "decompression_time": 0,#decompression_time,
            "loss_rate": loss_rate
        }
        self.results.append(result)

In [28]:
import abc

class CompressionAlgorithm(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def name(self):
        ...

    @abc.abstractmethod
    def compression(self, table):
        ...

    @abc.abstractmethod
    def decompression(self, compressed_data):
        ...

class LearningCompressionAlgorithm(CompressionAlgorithm):

    @abc.abstractmethod
    def fit(self, training_data):
        ...

class NonLearningCompressionAlgorithm(CompressionAlgorithm):
    ...

class LosslessCompressionAlgorithm(CompressionAlgorithm):
    ...

class LossyCompressionAlgorithm(CompressionAlgorithm):
    ...

In [29]:
class DummyCompressionAlgorithm(LosslessCompressionAlgorithm):
    name = 'Dummy Compression Algorithm'
    def compression(self, table):
        time.sleep(2)
        return table
    def decompression(self, compressed_data):
        time.sleep(1)
        return compressed_data

In [30]:
class Processing(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def __name__(self):
        ...

    @abc.abstractmethod
    def preprocess(self, raw_table):
        ...

    @abc.abstractmethod
    def postprocess(self, processed_table):
        ...

In [31]:
class DummyProcessing(Processing):
    name = "Dummy Pre/Post-Processing"
    def preprocess(self, raw_table):
        return raw_table
    def postprocess(self, processed_table):
        return processed_table


In [32]:
import numpy as np
    platform = Platform([DummyCompressionAlgorithm()], [np.array([1,2,3])])
    platform.compress_table()
    platform.results

ValueError: too many values to unpack (expected 2)